In [32]:
import os
import pandas as pd
import numpy as np
import math
import cmath
import matplotlib.pyplot as plt
import seaborn as sns

# Data

In [33]:
pd.set_option("display.max_columns", None)

In [30]:
df = pd.read_csv("../data/jetblue_df.csv")

In [34]:
df.head()

,Unnamed: 0,legId,searchDate,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,elapsedDays,isBasicEconomy,isRefundable,isNonStop,baseFare,totalFare,seatsRemaining,totalTravelDistance,segmentsDepartureTimeEpochSeconds,segmentsDepartureTimeRaw,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
0,5,79eda9f841e226a1e2121d74211e595c,2022-04-16,2022-04-17,ATL,BOS,VH0AUEL1,PT2H38M,0,False,False,True,217.67,248.60,7,947.0,1650206700,2022-04-17T10:45:00.000-04:00,1650216180,2022-04-17T13:23:00.000-04:00,BOS,ATL,JetBlue Airways,B6,NaN,9480,947,coach
1,29,24c07cf38de8a7a0ed2d7709168042a8,2022-04-16,2022-04-17,ATL,BOS,QH0KUEL1,PT2H44M,0,False,False,True,316.73,355.09,7,947.0,1650233220,2022-04-17T18:07:00.000-04:00,1650243060,2022-04-17T20:51:00.000-04:00,BOS,ATL,JetBlue Airways,B6,NaN,9840,947,coach
2,57,a291a2b5ac19223c3407cfec58d1fc24,2022-04-16,2022-04-17,ATL,CLT,VH0AUEL1,PT8H48M,0,False,False,False,482.79,542.60,7,1675.0,1650206700||1650229500,2022-04-17T10:45:00.000-04:00||2022-04-17T17:0...,1650216180||1650238380,2022-04-17T13:23:00.000-04:00||2022-04-17T19:3...,BOS||CLT,ATL||BOS,JetBlue Airways||JetBlue Airways,B6||B6,||Embraer 190,9480||8880,947||728,coach||coach
3,146,d53bcf9e8b6523fa2de9420d7cb22196,2022-04-16,2022-04-17,ATL,DTW,RH0AUEL1,PT5H8M,0,False,False,False,269.78,313.61,7,1247.0,1650235800||1650246600,2022-04-17T18:50:00.000-04:00||2022-04-17T21:5...,1650244380||1650254280,2022-04-17T21:13:00.000-04:00||2022-04-17T23:5...,JFK||DTW,ATL||JFK,JetBlue Airways||JetBlue Airways,B6||B6,Embraer 190||Embraer 190,8580||7680,762||485,coach||coach
4,209,5bb138f39804320ece767e747608a50e,2022-04-16,2022-04-17,ATL,EWR,VC0QBEL1,PT2H9M,0,False,False,True,171.16,198.60,7,762.0,1650195000,2022-04-17T07:30:00.000-04:00,1650202740,2022-04-17T09:39:00.000-04:00,EWR,ATL,JetBlue Airways,B6,Embraer 190,7740,762,coach


In [35]:
sorted(df.columns)

['Unnamed: 0',
 'baseFare',
 'destinationAirport',
 'elapsedDays',
 'fareBasisCode',
 'flightDate',
 'isBasicEconomy',
 'isNonStop',
 'isRefundable',
 'legId',
 'searchDate',
 'seatsRemaining',
 'segmentsAirlineCode',
 'segmentsAirlineName',
 'segmentsArrivalAirportCode',
 'segmentsArrivalTimeEpochSeconds',
 'segmentsArrivalTimeRaw',
 'segmentsCabinCode',
 'segmentsDepartureAirportCode',
 'segmentsDepartureTimeEpochSeconds',
 'segmentsDepartureTimeRaw',
 'segmentsDistance',
 'segmentsDurationInSeconds',
 'segmentsEquipmentDescription',
 'startingAirport',
 'totalFare',
 'totalTravelDistance',
 'travelDuration']

In [36]:
df.shape

(6824440, 28)

In [37]:
df.size

191084320

First need to address that the Unamed:0 column and save that as true. 

In [38]:
df.drop("Unnamed: 0", axis=1, inplace=True)

In [39]:
df.head()

,legId,searchDate,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,elapsedDays,isBasicEconomy,isRefundable,isNonStop,baseFare,totalFare,seatsRemaining,totalTravelDistance,segmentsDepartureTimeEpochSeconds,segmentsDepartureTimeRaw,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
0,79eda9f841e226a1e2121d74211e595c,2022-04-16,2022-04-17,ATL,BOS,VH0AUEL1,PT2H38M,0,False,False,True,217.67,248.60,7,947.0,1650206700,2022-04-17T10:45:00.000-04:00,1650216180,2022-04-17T13:23:00.000-04:00,BOS,ATL,JetBlue Airways,B6,NaN,9480,947,coach
1,24c07cf38de8a7a0ed2d7709168042a8,2022-04-16,2022-04-17,ATL,BOS,QH0KUEL1,PT2H44M,0,False,False,True,316.73,355.09,7,947.0,1650233220,2022-04-17T18:07:00.000-04:00,1650243060,2022-04-17T20:51:00.000-04:00,BOS,ATL,JetBlue Airways,B6,NaN,9840,947,coach
2,a291a2b5ac19223c3407cfec58d1fc24,2022-04-16,2022-04-17,ATL,CLT,VH0AUEL1,PT8H48M,0,False,False,False,482.79,542.60,7,1675.0,1650206700||1650229500,2022-04-17T10:45:00.000-04:00||2022-04-17T17:0...,1650216180||1650238380,2022-04-17T13:23:00.000-04:00||2022-04-17T19:3...,BOS||CLT,ATL||BOS,JetBlue Airways||JetBlue Airways,B6||B6,||Embraer 190,9480||8880,947||728,coach||coach
3,d53bcf9e8b6523fa2de9420d7cb22196,2022-04-16,2022-04-17,ATL,DTW,RH0AUEL1,PT5H8M,0,False,False,False,269.78,313.61,7,1247.0,1650235800||1650246600,2022-04-17T18:50:00.000-04:00||2022-04-17T21:5...,1650244380||1650254280,2022-04-17T21:13:00.000-04:00||2022-04-17T23:5...,JFK||DTW,ATL||JFK,JetBlue Airways||JetBlue Airways,B6||B6,Embraer 190||Embraer 190,8580||7680,762||485,coach||coach
4,5bb138f39804320ece767e747608a50e,2022-04-16,2022-04-17,ATL,EWR,VC0QBEL1,PT2H9M,0,False,False,True,171.16,198.60,7,762.0,1650195000,2022-04-17T07:30:00.000-04:00,1650202740,2022-04-17T09:39:00.000-04:00,EWR,ATL,JetBlue Airways,B6,Embraer 190,7740,762,coach


# Null Handling

In [40]:
print("Finding nulls per column")
df.isnull().any()

Finding nulls per column


legId                                False
searchDate                           False
flightDate                           False
startingAirport                      False
destinationAirport                   False
fareBasisCode                        False
travelDuration                       False
elapsedDays                          False
isBasicEconomy                       False
isRefundable                         False
isNonStop                            False
baseFare                             False
totalFare                            False
seatsRemaining                       False
totalTravelDistance                   True
segmentsDepartureTimeEpochSeconds    False
segmentsDepartureTimeRaw             False
segmentsArrivalTimeEpochSeconds      False
segmentsArrivalTimeRaw               False
segmentsArrivalAirportCode           False
segmentsDepartureAirportCode         False
segmentsAirlineName                  False
segmentsAirlineCode                  False
segmentsEqu

In [41]:
print("Observing null columns")
df[["totalTravelDistance", "segmentsEquipmentDescription", "segmentsDistance"]].head()

Observing null columns


,totalTravelDistance,segmentsEquipmentDescription,segmentsDistance
0,947.0,NaN,947
1,947.0,NaN,947
2,1675.0,||Embraer 190,947||728
3,1247.0,Embraer 190||Embraer 190,762||485
4,762.0,Embraer 190,762


In [42]:
null_col_sums = df.isna().sum()
display(null_col_sums)
null_numerators = [x for x in null_col_sums if x!= 0]

denominator = df.shape[0] #we are using the shape tuple to pull in the total rows to accurately understand how much is missing on a grand perspective

for null_numerator in null_numerators:
    print((null_numerator/denominator)*100)

legId                                    0
searchDate                               0
flightDate                               0
startingAirport                          0
destinationAirport                       0
fareBasisCode                            0
travelDuration                           0
elapsedDays                              0
isBasicEconomy                           0
isRefundable                             0
isNonStop                                0
baseFare                                 0
totalFare                                0
seatsRemaining                           0
totalTravelDistance                  56436
segmentsDepartureTimeEpochSeconds        0
segmentsDepartureTimeRaw                 0
segmentsArrivalTimeEpochSeconds          0
segmentsArrivalTimeRaw                   0
segmentsArrivalAirportCode               0
segmentsDepartureAirportCode             0
segmentsAirlineName                      0
segmentsAirlineCode                      0
segmentsEqu

0.8269689527638898
0.7684586574136486
0.2852834811354485


With the JetBlue data subset only showing around 0.82% of nulls existent, the executive decision that comes to mind is to remove the nulls. However, one thing to still do is to count the unique pairings of these elements to understand exactly what we are removing. 

In [43]:
#find a way to filter all unique pairings of T/F...
#does this warrant some feature engineering?

df["totalTravelDistance_null_existence"] =  df["totalTravelDistance"].isnull()
df["segmentsDistance_null_existence"] = df["segmentsDistance"].isnull()
df["segmentsEquipmentDescription_null_existence"] = df["segmentsEquipmentDescription"].isnull()

df.head()

,legId,searchDate,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,elapsedDays,isBasicEconomy,isRefundable,isNonStop,baseFare,totalFare,seatsRemaining,totalTravelDistance,segmentsDepartureTimeEpochSeconds,segmentsDepartureTimeRaw,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode,totalTravelDistance_null_existence,segmentsDistance_null_existence,segmentsEquipmentDescription_null_existence
0,79eda9f841e226a1e2121d74211e595c,2022-04-16,2022-04-17,ATL,BOS,VH0AUEL1,PT2H38M,0,False,False,True,217.67,248.60,7,947.0,1650206700,2022-04-17T10:45:00.000-04:00,1650216180,2022-04-17T13:23:00.000-04:00,BOS,ATL,JetBlue Airways,B6,NaN,9480,947,coach,False,False,True
1,24c07cf38de8a7a0ed2d7709168042a8,2022-04-16,2022-04-17,ATL,BOS,QH0KUEL1,PT2H44M,0,False,False,True,316.73,355.09,7,947.0,1650233220,2022-04-17T18:07:00.000-04:00,1650243060,2022-04-17T20:51:00.000-04:00,BOS,ATL,JetBlue Airways,B6,NaN,9840,947,coach,False,False,True
2,a291a2b5ac19223c3407cfec58d1fc24,2022-04-16,2022-04-17,ATL,CLT,VH0AUEL1,PT8H48M,0,False,False,False,482.79,542.60,7,1675.0,1650206700||1650229500,2022-04-17T10:45:00.000-04:00||2022-04-17T17:0...,1650216180||1650238380,2022-04-17T13:23:00.000-04:00||2022-04-17T19:3...,BOS||CLT,ATL||BOS,JetBlue Airways||JetBlue Airways,B6||B6,||Embraer 190,9480||8880,947||728,coach||coach,False,False,False
3,d53bcf9e8b6523fa2de9420d7cb22196,2022-04-16,2022-04-17,ATL,DTW,RH0AUEL1,PT5H8M,0,False,False,False,269.78,313.61,7,1247.0,1650235800||1650246600,2022-04-17T18:50:00.000-04:00||2022-04-17T21:5...,1650244380||1650254280,2022-04-17T21:13:00.000-04:00||2022-04-17T23:5...,JFK||DTW,ATL||JFK,JetBlue Airways||JetBlue Airways,B6||B6,Embraer 190||Embraer 190,8580||7680,762||485,coach||coach,False,False,False
4,5bb138f39804320ece767e747608a50e,2022-04-16,2022-04-17,ATL,EWR,VC0QBEL1,PT2H9M,0,False,False,True,171.16,198.60,7,762.0,1650195000,2022-04-17T07:30:00.000-04:00,1650202740,2022-04-17T09:39:00.000-04:00,EWR,ATL,JetBlue Airways,B6,Embraer 190,7740,762,coach,False,False,False


In [52]:
print(df.groupby(["totalTravelDistance_null_existence", "segmentsDistance_null_existence", "segmentsEquipmentDescription_null_existence"]).size())

print(f'The length of the df null existence series is: {len(df.groupby(["totalTravelDistance_null_existence", "segmentsDistance_null_existence", "segmentsEquipmentDescription_null_existence"]).size())}')

totalTravelDistance_null_existence  segmentsDistance_null_existence  segmentsEquipmentDescription_null_existence
False                               False                            False                                          6715947
                                                                     True                                             52057
True                                False                            False                                            36967
                                    True                             False                                            19083
                                                                     True                                               386
dtype: int64
The length of the df null existence series is: 5


From the top down, we have a series of the values where we grouped the column pairings of the True/False null existence features. The length is 5 and we're going to only select relevant features here. 

In [49]:
len(df.groupby(["totalTravelDistance_null_existence", "segmentsDistance_null_existence", "segmentsEquipmentDescription_null_existence"]).size())

5

In [50]:
df.groupby(["totalTravelDistance_null_existence", "segmentsDistance_null_existence", "segmentsEquipmentDescription_null_existence"]).size().sum()

6824440

In [11]:
#looking at when `totalTravelDistance` is null
print("Where totalTravelDistance is null")
df_but_null_total_dist = df[
    df["totalTravelDistance"].isnull()
][["totalTravelDistance", "segmentsDistance"]]
display(df_but_null_total_dist.head())


#looking at when `segmentsDistance` is null
print("Where segmentsDistance is null")
df_but_null_segment_dist = df[
    df["segmentsDistance"].isnull()
][["totalTravelDistance", "segmentsDistance"]]
display(df_but_null_segment_dist.head())


#looking at when `segmentsDistanc` and `totalTravelDistance` is null
print("Where segmentsDistance and totalTravelDistance is null")
df_but_null_total_and_segment_dist = df[
    (df["segmentsDistance"].isnull()) & 
    (df["segmentsDistance"].isnull())
][["totalTravelDistance", "segmentsDistance"]]
display(df_but_null_total_and_segment_dist.head())

Where totalTravelDistance is null


,totalTravelDistance,segmentsDistance
91367,NaN,NaN
941959,NaN,NaN
941960,NaN,NaN
941961,NaN,NaN
941962,NaN,NaN


Where segmentsDistance is null


,totalTravelDistance,segmentsDistance
91367,NaN,NaN
941959,NaN,NaN
941960,NaN,NaN
941961,NaN,NaN
941962,NaN,NaN


Where segmentsDistance and totalTravelDistance is null


,totalTravelDistance,segmentsDistance
91367,NaN,NaN
941959,NaN,NaN
941960,NaN,NaN
941961,NaN,NaN
941962,NaN,NaN
